### Делаем рекомендацию по хоррорам

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
links = pd.read_csv('../input/links.csv')
movies = pd.read_csv('../input/movies.csv')
ratings = pd.read_csv('../input/ratings.csv')
tags = pd.read_csv('../input/tags.csv')

In [3]:
links.head(3)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [4]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [5]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [6]:
tags.head(3)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992


### Выберем те фильмы у которых жанр только Horror, либо он стоит на первом месте

In [7]:
horror_movies_ = movies[(movies['genres'] == 'Horror') | 
                       (movies['genres'].apply(lambda x: x.split('|')[0]) == 'Horror')].reset_index()
horror_movies_ = horror_movies_.drop('index',axis=1)
horror_movies_

,movieId,title,genres
0,177,Lord of Illusions (1995),Horror
1,196,Species (1995),Horror|Sci-Fi
2,220,Castle Freak (1995),Horror
3,311,Relative Fear (1994),Horror|Thriller
4,328,Tales from the Crypt Presents: Demon Knight (1...,Horror|Thriller
5,332,Village of the Damned (1995),Horror|Sci-Fi
6,407,In the Mouth of Madness (1995),Horror|Thriller
7,426,Body Snatchers (1993),Horror|Sci-Fi|Thriller
8,512,"Puppet Masters, The (1994)",Horror|Sci-Fi
9,735,Cemetery Man (Dellamorte Dellamore) (1994),Horror


### Посчитаем количество тегов к каждому фильму и добавим это значение в нашу таблицу horror_movies_

In [13]:
horror_movies_tags = horror_movies_.merge(pd.DataFrame(tags.groupby('movieId').count()['tag']), on='movieId', how='left')
horror_movies_tags['tag'] = horror_movies_tags['tag'].apply(lambda x: int(0) if np.isnan(x) else int(x))
horror_movies_tags.head()

,movieId,title,genres,tag
0,177,Lord of Illusions (1995),Horror,0
1,196,Species (1995),Horror|Sci-Fi,0
2,220,Castle Freak (1995),Horror,0
3,311,Relative Fear (1994),Horror|Thriller,0
4,328,Tales from the Crypt Presents: Demon Knight (1...,Horror|Thriller,0


### Теперь разберёмся с рейтингами

In [14]:
movies_ratings =  horror_movies_.merge(ratings[['userId', 'movieId', 'rating']], on='movieId', how='left')

In [15]:
# посмотрим на ТОП-15 фильмов по количеству проставленных оценок и на их средний рейтинг
rating = movies_ratings.groupby('title').agg(['count', 'mean'])['rating'].sort_values('count', 
                                                                                  ascending=False)
rating.head(15)

,count,mean
title,,
Alien (1979),146,3.969178
"Shining, The (1980)",109,4.082569
Signs (2002),63,3.166667
"Exorcist, The (1973)",53,3.801887
Poltergeist (1982),47,3.425532
"Ring, The (2002)",47,3.202128
Species (1995),45,2.666667
"Birds, The (1963)",44,3.602273
Hollow Man (2000),39,2.294872


In [16]:
# посмотрим на медиану и среднее по количеству проставленных оценок на фильм и по их среднему
rating[['count', 'mean']].agg(['median', 'mean'])

,count,mean
median,2.000000,3.000000
mean,5.258547,2.793645


### Добавим к таблице с тегами данные по количеству проставленных оценок на фильм и средний рейтинг фильма

In [17]:
horror_movies = horror_movies_tags.merge(rating[['count', 'mean']], on='title', how='left')
horror_movies.head()

,movieId,title,genres,tag,count,mean
0,177,Lord of Illusions (1995),Horror,0,8,2.750000
1,196,Species (1995),Horror|Sci-Fi,0,45,2.666667
2,220,Castle Freak (1995),Horror,0,1,3.000000
3,311,Relative Fear (1994),Horror|Thriller,0,1,3.000000
4,328,Tales from the Crypt Presents: Demon Knight (1...,Horror|Thriller,0,10,3.150000


In [18]:
horror_movies.shape

(468, 6)

### Можно заметить что многие фильмы высоко оцененные достаточно старые, а новых встречается меньше, наверно это не очень хорошо. Теперь добавим год выпуска фильма в качестве признака

In [19]:
horror_movies['year'] = horror_movies['title'].apply(lambda x: x.split('(')[1][:-1])
horror_movies['year'] = horror_movies['year'].apply(lambda x: int(x) if len(x)==4 and x.isdigit() else 0)

In [20]:
horror_movies['year'].value_counts().head(10)

0       45
2007    15
2014    14
2009    14
2006    13
2013    12
2012    12
2011    12
2002    11
1980    11
Name: year, dtype: int64

In [21]:
horror_movies.head()

,movieId,title,genres,tag,count,mean,year
0,177,Lord of Illusions (1995),Horror,0,8,2.750000,1995
1,196,Species (1995),Horror|Sci-Fi,0,45,2.666667,1995
2,220,Castle Freak (1995),Horror,0,1,3.000000,1995
3,311,Relative Fear (1994),Horror|Thriller,0,1,3.000000,1994
4,328,Tales from the Crypt Presents: Demon Knight (1...,Horror|Thriller,0,10,3.150000,1995


### Выводим конечный рейтинг.
1. С подобранными мною "на глаз" коэффициентами взвесим теги, количество оценок, и средний рейтинг по фильму.
2. Разделим нашу таблицу на три: по году выхода фильма и будем поочередно выдавать лучший результат

In [22]:
alpha = 0.05
beta = 1
gamma = 0.5

horror_movies['rating_finish'] = horror_movies['count'] * alpha + horror_movies['mean'] * beta + \
                                    horror_movies['tag'] * gamma

horror_movies = horror_movies.sort_values('rating_finish', ascending=False)
horror_movies.head(10)

,movieId,title,genres,tag,count,mean,year,rating_finish
20,1258,"Shining, The (1980)",Horror,9,109,4.082569,1980,14.032569
19,1214,Alien (1979),Horror|Sci-Fi,1,146,3.969178,1979,11.769178
279,8957,Saw (2004),Horror|Mystery|Thriller,7,33,3.181818,2004,8.331818
76,1997,"Exorcist, The (1973)",Horror|Mystery,1,53,3.801887,1973,6.951887
220,5502,Signs (2002),Horror|Sci-Fi|Thriller,1,63,3.166667,2002,6.816667
31,1333,"Birds, The (1963)",Horror|Thriller,1,44,3.602273,1963,6.302273
73,1994,Poltergeist (1982),Horror|Thriller,1,47,3.425532,1982,6.275532
204,4725,Session 9 (2001),Horror|Thriller,4,4,3.625000,2001,5.825000
15,968,Night of the Living Dead (1968),Horror|Sci-Fi|Thriller,1,28,3.750000,1968,5.650000
225,5679,"Ring, The (2002)",Horror|Mystery|Thriller,0,47,3.202128,2002,5.552128


In [23]:
horror_rating_old = horror_movies[horror_movies['year'] < 1993]
horror_rating_middle = horror_movies[(horror_movies['year'] >= 1993) & (horror_movies['year'] < 2008)]
horror_rating_new = horror_movies[horror_movies['year'] >= 2008]

In [24]:
finish_list = []

for i, j, k in zip(horror_rating_old['title'], horror_rating_middle['title'], horror_rating_new['title']):
    finish_list.append(i)
    finish_list.append(j)
    finish_list.append(k)
    
finish_list[:15]

['Shining, The (1980)',
 'Saw (2004)',
 'Bone Tomahawk (2015)',
 'Alien (1979)',
 'Signs (2002)',
 'Conjuring, The (2013)',
 'Exorcist, The (1973)',
 'Session 9 (2001)',
 'Kill List (2011)',
 'Birds, The (1963)',
 'Ring, The (2002)',
 'Spirit Camp (2009)',
 'Poltergeist (1982)',
 'Pitch Black (2000)',
 'It Comes at Night (2017)']